In [1]:
import pygame
import random
import os

#initialise pygame
pygame.init()

#game screen dimensions
SCREEN_WIDTH = 500
SCREEN_HEIGHT = 700

#create game screen
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption('ASSIGNMENT 3 GAME')

#set frame rate
clock = pygame.time.Clock()
FPS = 60

#game variables
SCROLLING = 150
GRAVITY = 1
MAX_PLATFORMS = 10
scroll = 0
bg_scroll = 0
game_over = False
score = 0
counter = 0

if os.path.exists('score.txt'):
    with open('score.txt', 'r') as file:
         high_score = int(file.read())
else:
    high_score = 0

#define colors
WHITE = (255, 255, 255)
YELLOW = (255,250, 205)
BLACK = (0, 0, 0)
PANEL = (217, 242, 228)

#define font
font_small = pygame.font.SysFont(None, 20)
font_big = pygame.font.SysFont(None, 30)

#load images
hero_img = pygame.image.load('hero.png').convert_alpha()
background_img = pygame.image.load('game_background.png').convert_alpha()
platform_img = pygame.image.load('platform.png').convert_alpha()


#function for outputting text onto the screen
def draw_text(text, font, text_col, x, y):
    img = font.render(text, True, text_col)
    screen.blit(img, (x, y))

#function for drawing info panel
def draw_panel():
    pygame.draw.rect(screen, PANEL, (0, 0, SCREEN_WIDTH, 30))
    pygame.draw.line(screen, BLACK, (0, 30), (SCREEN_WIDTH, 30), 2)
    draw_text('SCORE: ' + str(score), font_big, BLACK, 200, 8)
    
#function for drawing the background
def draw_bg(bg_scroll):
    screen.blit(background_img, (0, 0 + bg_scroll))
    screen.blit(background_img, (0, -800 + bg_scroll))
    
    
    
#player class
class Player():
    def __init__(self, x, y):
        self.image = hero_img
        self.width = 30
        self.height = 60
        self.rect = pygame.Rect(0, 0, self.width, self.height)
        self.rect.center = (x, y)
        self.vel_y = 0
        self.flip = False
        
    def move(self):
        #reset variables
        scroll = 0
        dx = 0
        dy = 0
        
        
        #process keypress
        key = pygame.key.get_pressed()
        if key[pygame.K_LEFT]:
            dx = -10
            self.flip = True
        if key[pygame.K_RIGHT]:
            dx = 10
            self.flip = False
            
        #gravity
        self.vel_y += GRAVITY
        dy += self.vel_y
            
        #ensure hero doesn't go off the edge of screen
        if self.rect.left + dx < 0:
            dx = -self.rect.left
        if self.rect.right + dx > SCREEN_WIDTH:
            dx = SCREEN_WIDTH - self.rect.right
            
        #check collision w/ platforms
        for platform in platform_group:
            #collision in the y direction
            if platform.rect.colliderect(self.rect.x, self.rect.y + dy, self.width, self.height):
                #check if above the platform
                if self.rect.bottom < platform.rect.centery:
                    if self.vel_y > 0:
                        self.rect.bottom = platform.rect.top
                        dy = 0
                        self.vel_y = -23
            

        #check if the hero has bounced to the top of the screen
        if self.rect.top <= SCROLLING:
            #if player is jumping
            if self.vel_y <0:
                scroll = -dy
            
        #update rectangle pos
        self.rect.x += dx
        self.rect.y += dy + scroll
        
        return scroll
        
    def draw(self):
        screen.blit(pygame.transform.flip(self.image, self.flip, False), (self.rect.x - 15, self.rect.y -13))
        pygame.draw.rect(screen, WHITE, self.rect, 2)
        
#platform class
class Platform(pygame.sprite.Sprite):
    def __init__(self, x, y, width, moving):
        pygame.sprite.Sprite.__init__(self)
        self.image = pygame.transform.scale(platform_img, (width, 10))
        self.moving = moving
        self.move_counter = random.randint(0, 50)
        self.direction = random.choice([-1, 1])
        self.speed = random.randint(1, 2)
        self.rect = self.image.get_rect()
        self.rect.x = x
        self.rect.y = y
        
    def update(self, scroll):
        #moving platform s2s if it is a moving platform
        if self.moving == True:
            self.move_counter +=1
            self.rect.x += self.direction * self.speed
            
        #change platform direction if it has moved fully or hit a wall
        if self.move_counter >= 100 or self.rect.left < 0 or self.rect.right > SCREEN_WIDTH:
            self.direction *= -1
            self.move_counter = 0
        
        #update platform's vertical position
        self.rect.y += scroll
        
        #check if platform has gone off the screen
        if self.rect.top > SCREEN_HEIGHT:
            self.kill()
        
        
        
#hero instance        
hero = Player(SCREEN_WIDTH // 2, SCREEN_HEIGHT -150)

#create sprite groups
platform_group = pygame.sprite.Group()

#create starting platform
platform = Platform(SCREEN_WIDTH // 2 -50, SCREEN_HEIGHT -50, 100, False)
platform_group.add(platform)  
        
#loop
run = True
while run:
    
    clock.tick(FPS)
    
    if game_over == False:
        scroll = hero.move()
    
        #draw background
        bg_scroll += scroll
        if bg_scroll >= 1000:
            bg_scroll = 0
        draw_bg(bg_scroll)
    
        #generate platforms
        if len(platform_group) < MAX_PLATFORMS:
            p_w = random.randint(40, 60)
            p_x = random.randint(0, SCREEN_WIDTH - p_w)
            p_y = platform.rect.y - random.randint(80, 120)
            p_type = random.randint(1, 2)
            if p_type == 1 and score > 500:
                p_moving = True
            else:
                p_moving = False
            platform = Platform(p_x, p_y, p_w, p_moving)
            platform_group.add(platform)
        
    
        #update platforms
        platform_group.update(scroll)
        
        
        #update score
        if scroll > 0:
            score +=scroll
        
        #draw line at previous high score
        pygame.draw.line(screen, BLACK, (0, score - high_score + SCROLLING), (SCREEN_WIDTH, score - high_score + SCROLLING), 2)
        draw_text('HIGH SCORE', font_big, BLACK, SCREEN_WIDTH - 150, score - high_score + SCROLLING)
        
        #draw sprites
        platform_group.draw(screen)
        hero.draw()
        
        #draw panel
        draw_panel()
    
        #check game over
        if hero.rect.top > SCREEN_HEIGHT:
            game_over = True
    
    else:
        if counter < SCREEN_WIDTH:
            counter += 5
            pygame.draw.rect(screen, YELLOW, (0, 0, 600, SCREEN_HEIGHT))
        draw_text('GAME OVER!', font_big, BLACK, 190, 250)
        draw_text('SCORE: ' + str(score), font_big, BLACK, 200, 300)
        draw_text('PRESS SPACE TO PLAY AGAIN', font_big, BLACK, 110, 350)
        draw_text('HIGH SCORE: ' + str(high_score), font_big, BLACK, 170, 400)
        
        #update high score
        if score > high_score:
            high_score = score
            with open('score.txt', 'w') as file:
                file.write(str(high_score))
        key = pygame.key.get_pressed()
        if key[pygame.K_SPACE]:
            #reset variable
            game_over = False
            score = 0
            scroll = 0
            counter = 0
            
            
            #reposition hero
            hero.rect.center = (SCREEN_WIDTH // 2, SCREEN_HEIGHT -150)
            #reset platforms
            platform_group.empty()
            ##create starting platform
            platform = Platform(SCREEN_WIDTH // 2 -80, SCREEN_HEIGHT -50, 100, False)
            platform_group.add(platform)
            
    
    #event handler
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            run = False
            
            
    #update dispay screen
    pygame.display.update()
            

pygame.quit()



pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
